In [2]:
import pandas as pd
import numpy as np

# Procesamiento columna 'Descripción'

In [3]:
excel_file = pd.ExcelFile('../data/raw/data.xlsx')
sheets = excel_file.sheet_names
sheets

['Lista de equipos',
 'Historico calif 2_1(10 años)',
 'T-PAP(C)_T-ACE',
 'ARC(I)_ARC(D)',
 'DP',
 'OIL.TAP',
 'DIEL.PF_ DIEL',
 'OIL.CORR',
 'PRO-M',
 'M.OLTC']

In [4]:
import unidecode

def norm_descrpcion(s):
    s = s.lower()
    s = unidecode.unidecode(s)
    return s

converter_descripcion = {}
converter_descripcion['Descripción'] = norm_descrpcion

df_collection = {}
for name in sheets:
    df_collection[name] = pd.read_excel('../data/raw/data.xlsx', sheet_name=name, converters=converter_descripcion)

1. Separar cada fila en cada una de las descripciones. i.e : "realizar analisis dga, humedad y furanos" deben quedar dos filas exactamente iguales pero con cada uno de los registros. -> pd.explode(), explode a esa accion de expandir un dataset con base a una regla.

2. Aplicar un algoritmo de distancia de levehnstein. 

3. Análisis descriptivo que relacione cada equipo con las acciones. i.e: frecuencias de cada accion para cada equipo.

## Separación manual

In [26]:
def separacion_manual(dframe, originales, reemplazos, adiciones):
    df = dframe.copy()
    for i in range(len(originales)):
        index = df['Descripción'].index(originales[i])
        if sum(index) > 0:
            index = index[index]
            index = index[0]
            instancias = df.loc[index].copy()
            df.loc[index,'Descripción'] = reemplazos[i]
            df.append(instancias, ignore_index = True)
            df.loc[-1,'Descripción'] = adiciones[i]
    return df


df_collection_manuales = {}
converter_manuales = {}
converter_manuales['Originales'] = norm_descrpcion
converter_manuales['Reemplazos'] = norm_descrpcion
converter_manuales['Adiciones'] = norm_descrpcion

df_collection_separados = {}
#for name in sheets:
    #df_collection_separados[name] = df_collection[name].copy()

excel_file_reemplazos = pd.ExcelFile('../data/reemplazos_manual/reemplazos_manual.xlsx')
sheets_reemplazos = excel_file_reemplazos.sheet_names

df = {}
for name in ['T-PAP(C)_T-ACE']:
    df_collection_manuales[name] = pd.read_excel('../data/reemplazos_manual/reemplazos_manual.xlsx', sheet_name = name, converters = converter_manuales)
    index_manuales = ~df_collection_manuales[name].loc[:,'Reemplazos'].isna()
    originales = df_collection_manuales[name].loc[index_manuales, 'Originales'].values
    reemplazos = df_collection_manuales[name].loc[index_manuales, 'Reemplazos'].values
    adiciones = df_collection_manuales[name].loc[index_manuales, 'Adiciones'].values
    #df_collection_separados[name] = separacion_manual(df_collection[name], originales, reemplazos, adiciones)
    df[name] = df_collection[name].copy()
    for i in [0]:#range(len(originales)):
        index = df[name]['Descripción'] == originales[i]
        if sum(index) > 0:
            #Sólo tener en cuenta el primero
            for j in range(len(index)):
                if index[j] == True:
                    index[j + 1:] = False
                    break
                    
            instancia = df[name].loc[index].copy()
            df[name].loc[index,'Descripción'] = reemplazos[i]
            print(len(df[name]))
            df[name].append(instancia, ignore_index = True)
            print(len(df[name]))
            df[name].loc[len(df[name]) - 1,'Descripción'] = adiciones[i]


 
 
# df_collection_manuales['T-PAP(C)_T-ACE'] = pd.read_excel('../data/reemplazos_manual/reemplazos_manual.xlsx', sheet_name = 'T-PAP(C)_T-ACE', converters = converter_manuales)
# index_manuales = ~df_collection_manuales['T-PAP(C)_T-ACE'].loc[:,'Reemplazos'].isna()
# originales = df_collection_manuales['T-PAP(C)_T-ACE'].loc[index_manuales, 'Originales'].values
# reemplazos = df_collection_manuales['T-PAP(C)_T-ACE'].loc[index_manuales, 'Reemplazos'].values
# adiciones = df_collection_manuales['T-PAP(C)_T-ACE'].loc[index_manuales, 'Adiciones'].values
# df_collection_separados[name] = separacion_manual(df_collection[name], originales, reemplazos, adiciones)

# index = df_collection['T-PAP(C)_T-ACE']['Descripción'] == originales[0]

#df_collection_separados[name] = separacion_manual(df_collection_separados[name], originales, reemplazos, adiciones)

df_collection['T-PAP(C)_T-ACE'].tail(5)

156
156


,Sociedad CO,Equipo,Denominación,Perfil catálogo,Ubicac.técnica,Indicador ABC,Área de empresa,Pto.tbjo.resp.,Clase de aviso,Aviso,Descripción,Creado el,Inicio deseado,Fin deseado,Prioridad,Fecha de cierre,Status usuario,Status sistema,Autor del aviso
8,ISA,119040,Autotrafo Monofásico 500/230 kV-B,N-AUTOTR,CERR500ATR-3-AUTOT,NaN,NOR,38B40005,N4,236078,muestra de aceite y analisis dga,2011-05-16,2011-06-29,2011-06-29,2.0,2011-07-07,INIC COND,MECE,FAHERRENO


In [6]:
# Extrayendo valores únicos:
# unicos_descripcion = {}

# for name in sheets:
#     if 'Descripción' in df_collection[name].columns:
#         unicos_descripcion[name] = pd.unique(df_collection[name]['Descripción'])
#         #print(name + ' ' + unicos_descripcion[name])


# unicos_descripcion['T-PAP(C)_T-ACE']